In [19]:
import pandas as pd
import requests

In [20]:
df = pd.read_csv('funders.csv')
df

,name,acronym,country,name_variant,type
0,National Institutes of Health,NIH,United States,NaN,Funder
1,National Science Foundation,NSF,United States,NaN,Funder
2,Natural Sciences and Engineering Research Council,NSERC,Canada,NaN,Funder
3,Bill & Melinda Gates Foundation,Gates Foundation,United States,BMGF,Charity
4,Social Sciences and Humanities Research Council,SSHRC,Canada,NaN,Funder
...,...,...,...,...,...
195,Fondation Leducq,NaN,France,NaN,Funder
196,Association Française contre les Myopathies,AFM,France,NaN,Funder
197,Swedish Foundation for Strategic Research,SFSR,Sweden,NaN,Funder
198,Total Foundation,NaN,France,NaN,Funder


In [24]:
country_mapping= {
        'United States': 'US', 'Canada': 'CA', 'Netherlands': 'NL',
        'Australia': 'AU', 'Japan': 'JP', 'France': 'FR',
        'Sweden': 'SE', 'Switzerland': 'CH', 'United Kingdom': 'GB',
        'Germany': 'DE', 'Brazil': 'BR', 'European Union': 'EU',
        'Finland': 'FI', 'Denmark': 'DK', 'Portugal': 'PT',
        'Spain': 'ES', 'Italy': 'IT', 'Poland': 'PL',
        'Austria': 'AT', 'Czech Republic': 'CZ', 'China': 'CN',
        'South Korea': 'KR', 'Taiwan': 'TW', 'India': 'IN',
        'Argentina': 'AR', 'Chile': 'CL', 'Israel': 'IL',
        'Turkey': 'TR', 'Colombia': 'CO', 'Mexico': 'MX',
        'Peru': 'PE', 'Kenya': 'KE', 'Nigeria': 'NG',
        'Malaysia': 'MY', 'Thailand': 'TH', 'Vietnam': 'VN',
        'International': 'INT', 'Qatar': 'QA', 'Uruguay': 'UY',
        'South Africa': 'ZA', 'Ecuador': 'EC', 'Costa Rica': 'CR',
        'Uganda': 'UG', 'Tanzania': 'TZ', 'International (Africa)': 'AFR',
        'Indonesia': 'ID', 'Philippines': 'PH', 'Pakistan': 'PK',
        'Bangladesh': 'BD', 'Sri Lanka': 'LK', 'Nepal': 'NP',
    'Russia': 'RU'
    }

def search_openalex(query, country_code=None, verbose=False):
    """
    Recherche un funder dans l'API OpenAlex
    
    Args:
        query: Terme de recherche (nom ou acronyme)
        country_code: Code pays pour filtrer (optionnel)
    
    Returns:
        dict ou None: Informations du funder trouvé
    """
    url = "https://api.openalex.org/funders"
    params = {
        'search': query,
        'per_page': 5,
        'mailto': 'your-email@example.com'  # Recommandé par OpenAlex
    }

    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if data.get('results'):
            
            if verbose:
                print(pd.DataFrame(data['results']))
            
            # Si on a un code pays, chercher correspondance exacte
            if country_code:
                for result in data['results']:
                    if isinstance(result.get('country_code'), str):
                        result_country = result.get('country_code', '').upper()
                        if result_country == country_code.upper():
                            return result
            # Sinon, prendre le premier résultat
            return data['results'][0]
        return None

    except requests.exceptions.RequestException as e:
        print(f"  ⚠️  Erreur réseau: {e}")
        return None
    except Exception as e:
        print(f"  ⚠️  Erreur: {e}")
        return None


In [25]:
data = df.to_dict(orient='records')
len(data)

200

In [26]:
for d in data:
    country_code = country_mapping.get(d['country'], '')
    search_query = f"{d['name']}"
    result = search_openalex(search_query, country_code)
    if result:
        for f in ['id', 'display_name', 'alternate_titles', 'description', 'homepage_url', 'image_thumbnail_url' ]:
            d[f'openalex_{f}'] = result[f]
        if 'ids' in result:
            for t in ['ror', 'wikidata']:
                if t in result['ids']:
                    d[t] = result['ids'][t]

In [27]:
df2 = pd.DataFrame(data)
df2.to_csv('funders_with_openalex_id.csv', index=False)

In [28]:
len(df2)

200

In [29]:
df2

,name,acronym,country,name_variant,type,openalex_id,openalex_display_name,openalex_alternate_titles,openalex_description,openalex_homepage_url,openalex_image_thumbnail_url,ror,wikidata
0,National Institutes of Health,NIH,United States,NaN,Funder,https://openalex.org/F4320332161,National Institutes of Health,"[US National Institutes of Health, Institutos ...",US government medical research agency,http://www.nih.gov,https://commons.wikimedia.org/w/index.php?titl...,https://ror.org/01cwqze88,https://www.wikidata.org/entity/Q390551
1,National Science Foundation,NSF,United States,NaN,Funder,https://openalex.org/F4320306076,National Science Foundation,"[U.S. National Science Foundation, USA NSF, US...",United States government agency,https://www.nsf.gov,https://commons.wikimedia.org/w/index.php?titl...,https://ror.org/021nxhr62,https://www.wikidata.org/entity/Q304878
2,Natural Sciences and Engineering Research Council,NSERC,Canada,NaN,Funder,https://openalex.org/F4320334593,Natural Sciences and Engineering Research Coun...,"[CRSNG, NSERC, Conseil de Recherches en Scienc...",agency of the Government of Canada that provid...,http://www.nserc-crsng.gc.ca/index_eng.asp,None,https://ror.org/01h531d29,https://www.wikidata.org/entity/Q2993807
3,Bill & Melinda Gates Foundation,Gates Foundation,United States,BMGF,Charity,https://openalex.org/F4320306137,Bill and Melinda Gates Foundation,"[Gates Foundation, The Bill & Melinda Gates Fo...",private foundation founded by Bill and Melinda...,https://www.gatesfoundation.org,https://commons.wikimedia.org/w/index.php?titl...,https://ror.org/0456r8d26,https://www.wikidata.org/entity/Q655286
4,Social Sciences and Humanities Research Council,SSHRC,Canada,NaN,Funder,https://openalex.org/F4320334617,Social Sciences and Humanities Research Counci...,[Conseil de recherches en sciences humaines du...,Canadian funding body,http://www.sshrc-crsh.gc.ca,None,https://ror.org/04j5jqy92,https://www.wikidata.org/entity/Q2993802
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Fondation Leducq,NaN,France,NaN,Funder,https://openalex.org/F4320320890,Fondation Leducq,[Leducq Foundation],None,https://www.fondationleducq.org,None,https://ror.org/01czwga19,https://www.wikidata.org/entity/Q6512782
196,Association Française contre les Myopathies,AFM,France,NaN,Funder,https://openalex.org/F4320309751,Association Française contre les Myopathies,"[French Association against Myopathies, French...",French patient association,http://www.afm-telethon.com,None,https://ror.org/0162y2387,https://www.wikidata.org/entity/Q2867701
197,Swedish Foundation for Strategic Research,SFSR,Sweden,NaN,Funder,https://openalex.org/F4320320940,Stiftelsen för Strategisk Forskning,"[SSF, Swedish Foundation for Strategic Research]",None,http://www.stratresearch.se/en,https://commons.wikimedia.org/w/index.php?titl...,https://ror.org/044wr7g58,https://www.wikidata.org/entity/Q7654688
198,Total Foundation,NaN,France,NaN,Funder,https://openalex.org/F4320329988,Total Foundation,[],"primary school in Ogun State, Nigeria",None,None,None,https://www.wikidata.org/entity/Q116427110


In [ ]:
df2.head(40)

In [30]:
df2[pd.isnull(df2.openalex_id)]

,name,acronym,country,name_variant,type,openalex_id,openalex_display_name,openalex_alternate_titles,openalex_description,openalex_homepage_url,openalex_image_thumbnail_url,ror,wikidata
85,Ministry of Higher Education Malaysia Grants S...,MyGRANTS,Malaysia,MOHE,Funder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,Hubert Curien Partnerships,PHC,France,Partenariat Hubert Curien,Funder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,Hospital Program for Clinical Research,PHRC,France,Programme Hospitalier de Recherche Clinique,Funder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,France 2030 Investment Plan,PIA,France,Programme d’Investissements d’Avenir|Investiss...,Funder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,Région Ile de France,NaN,France,Région IdF,Funder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,Région Grand Est,NaN,France,Région Alsace|Région Lorraine,Funder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,Région Centre-Val de Loire,NaN,France,Région centre,Funder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,Région Bourgogne Franche-Comté,NaN,France,Région Bourgogne|Regional Council of Burgundy,Funder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,BPI France,NaN,France,BPI|BPIFrance,Funder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,Sigrid Juselius Foundation,NaN,Finland,NaN,Funder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
